In [14]:
# @TODO Dynamically filter peptide set based on length(s) of input sequences of binders
#       i.e. 2 binders, one 11 AA long, one 13 AA long, each gets their own "subset" of the
#       full peptide lilst that can be compared to it. For any number of input sequences

# @TODO implement method for both similarities to M6 and GrBP5 to interrelate and act as their own feature set:
# i.e. if a peptide matches both peptides in temrs of sequence at some index, that should be important rather than
# having it equal to one matching only one

import pandas as pd
import numpy as np
from scipy.stats import kendalltau
import matplotlib.pyplot as plt
from typing import Set, Tuple, Dict, List

class SequenceSimilarity:
    '''
    Class that takes in a path to a list of amino acid sequences as well
    as any number of peptide sequences explicitly that are known to have
    a certain set of properties. Generates metrics for similarity for each
    peptide in path and returns domains AA sequence with high similarity
    '''

    def __init__(self, binders: Dict, data_paths: Dict, peps_path: str, aa_col: str):
        
        self.AA = set('LINGVEPHKAYWQMSCTFRD')
        self.binder_dict = binders
        self.data_paths_dict = data_paths
        self.__update_binders()
        self.__update_similarity_data()

        self.aa_col = aa_col
        self.columns = ['EIIP_Seq', 'Num_Seq', 'PAM30', 'BLOSUM', 'RRM_SN', 'RRM_Corr', 'Matching_sseqs']

        self.peps = dict.fromkeys(self.binders)
        self.peps=pd.read_csv(peps_path)
        self.peps.columns = [aa_col]
        self.peps_same_len = self.peps[self.peps[aa_col].str.len() == self.binder_len]

        self.peps_sl_sim = self.peps_same_len.copy()
        for col in self.columns:
            self.peps_sl_sim[col] = None
        self.__get_similarities()

        self.sseq_set: Set[Tuple[int, str]] # full set of binder subseqs
        self.top_sseq: Set[Tuple[int, str]] # set of sub sequences w/ high simil
        self.top_seq: Set[str]              # set of peptides with high simil
    
    def __update_binders(self) -> None:
        '''
        Private method to set the binders stored by class and check to 
        make sure they are of the same length as one another
        @TODO Allow for different lengthed binders, creates different lengthd
        peptide dataframes for analysis
        '''
        print(self.binder_dict)
        self.binders = [binder for binder in self.binder_dict.values()]
        try:
            self.binder_len = len(self.binders[0])
        except:
            print("Need at least one input binder")
        
        for binder in self.binders:
            if len(binder) != self.binder_len:
                print("All binders must be of same length")
                # @TODO Handle multiple lengths of binders, each differently
                #       lengthed binder is compared with different parts of the
                #       full peptide set of the same length

    def __update_similarity_data(self) -> None:
        """
        Private method to store the paths of any data needed
        for similarity calcs and create Dataframes from them
        """
        self.data_paths: Dict = self.data_paths_dict
        self.data = dict.fromkeys(self.data_paths.keys())
        for data in self.data_paths.keys():
            if data == "AA_MAP":
                self.data[data] = pd.read_csv(self.data_paths[data])
            else:
                self.data[data] = pd.read_csv(self.data_paths[data], header=None)

    def __get_similarities(self) -> None:
        self._get_AA_conversion(conv_type='EIIP')
        self._get_AA_conversion(conv_type='Num')
        self._get_BLOSUM_similarity()
        self._get_PAM30_similarity()
        self._get_RRM_SN_similarity()
        self._get_RRM_corr_similarity()
        self._get_matching_sseqs()

    def _get_AA_conversion(self, conv_type: str = None) -> None:
        AA_map = self.data["AA_MAP"][['AA', conv_type ]]
        def get_aa_conv(pep):
            new_seq = list()
            for AA in pep:
                if AA not in self.AA:
                    val = 0
                else:
                    val = AA_map.loc[AA_map['AA']==AA][conv_type].values[0]
                new_seq.append(val)
            return new_seq
        # get_aa_conv = lambda pep: map(lambda aa: AA_map.loc[AA_map['AA']==aa][conv_type].values[0], pep)
        self.peps_sl_sim[conv_type+"_Seq"] = self.peps_sl_sim[self.aa_col].apply(get_aa_conv)

    def _get_PAM30_similarity(self) -> pd.DataFrame:
        pass

    def _get_BLOSUM_similarity(self) -> pd.DataFrame:
        pass

    def _get_RRM_SN_similarity(self) -> None:
        pass

    def _get_RRM_corr_similarity(self) -> None:
        pass

    def _get_matching_sseqs(self):

        pass

    def df_filter_subseq(self, sub_seq: str, ind: int = None) -> pd.DataFrame:
        '''
        Takes in a subsequence of equal or lesser length to
        peptides in class peptide dataframe and returns a dataframe
        containing only those peptides containing the sequence
        '''
        if not {*sub_seq}.issubset({*self.AA}):
            raise Exception('Invalid subsequence')
        if ind is None:
            return self.peps_sl_sim[self.peps_sl_sim[self.aa_col].str.contains(sub_seq)]
        return self.peps_sl_sim[self.peps_sl_sim[self.aa_col].str.find(sub_seq) == ind]

    def get_sim_matrix(self, seq) -> pd.DataFrame:
        return self.data.filter

    def get_binder_subseq(self) -> Dict[str, List[Tuple[str, int]]]: #Each binder associated with list of (sseq, index)
        '''
        Generates all possible subsequences for binders. Returns in dict, where
        each binder corresponds to a dictionary of the index where it occurs,
        and the subsequence that occurs
        '''
        all_sseq = lambda s: [(s[i:j], i) for i in range(len(s)) for j in range(i+1, len(s)+1)]
        sseq = dict.fromkeys(self.binders)
        for binder in sseq.keys():
            sseq[binder] = all_sseq(binder)
        return sseq


    def get_df_with_binder_subseqs(self, min_length: int = 0) -> Dict[str, pd.DataFrame]:
        '''
        Returns a filtered version of self.peps_same_len DataFrame containing only
        those rows with sequences which contain subsequences (of min length specified in parameter) 
        of the two binder sequences in the locations where they occur in the binders
        '''
        data: Dict[str, List[pd.DataFrame]] = dict.fromkeys(self.binders)
        sseq = self.get_binder_subseq()
        for binder in self.binders:
            filtered_data = []
            for (ss, i) in sseq[binder]:
                if len(ss) >= min_length:
                    filtered_data.append(self.df_filter_subseq(ss, i))
            data[binder] = pd.concat(filtered_data)
        return data

    def get_kendalltau_corr_map(self) -> Tuple:
        return kendalltau(self.data['AA_MAP'][['Num']], self.data['AA_MAP'][['EIIP']])

In [15]:
DATA_PATHS = {
        "BLOSUM":"../src_data/BLOSUM.csv",
        "PAM30":"../src_data/pam30.csv",
        "AA_MAP":"../src_data/aa_chart.csv",
}
SEQS = {
    'GRBP5':'IMVTESSDYSSY',
    'M6':'IMVTASSAYDDY'
}
AA_COL = 'Sequences'
PEP_PATH = '../src_data/Sequence_data.csv'
similarity = SequenceSimilarity(SEQS, DATA_PATHS, PEP_PATH, AA_COL)

    # --------------------------- debug
    # Check to make sure df filter works
print(similarity.df_filter_subseq('SVP', ind=0)) # it works

{'GRBP5': 'IMVTESSDYSSY', 'M6': 'IMVTASSAYDDY'}
          Sequences                                           EIIP_Seq  \
13     SVPHFSDEDKDP  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
28     SVPHFSEEEKEA  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
43     SVPHFSDEDKDP  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
58     SVPHFSEEEKEP  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
73     SVPHFSDEDKDP  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
13697  SVPVEPEDDDMV  [0.0829, 0.0057, 0.0198, 0.0057, 0.0058, 0.019...   
13760  SVPLRPEEDELI  [0.0829, 0.0057, 0.0198, 0.0, 0.0959, 0.0198, ...   
27635  SVPHFSDEDKDP  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
28199  SVPHFSEEEKEA  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
28341  SVPHFSDEDKDP  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
28788  SVPHFSEEEKEP  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   

                                    Num_Seq PAM30 BLOSUM RRM_SN

In [16]:
similarity

In [17]:
dir(similarity)

['AA',
 '_SequenceSimilarity__get_similarities',
 '_SequenceSimilarity__update_binders',
 '_SequenceSimilarity__update_similarity_data',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_AA_conversion',
 '_get_BLOSUM_similarity',
 '_get_PAM30_similarity',
 '_get_RRM_SN_similarity',
 '_get_RRM_corr_similarity',
 '_get_matching_sseqs',
 'aa_col',
 'binder_dict',
 'binder_len',
 'binders',
 'columns',
 'data',
 'data_paths',
 'data_paths_dict',
 'df_filter_subseq',
 'get_binder_subseq',
 'get_df_with_binder_subseqs',
 'get_kendalltau_corr_map',
 'get_sim_matrix',
 'peps',
 'peps_same_len',
 'peps_sl_sim']

In [26]:
similarity.peps_sl_sim

,Sequences,EIIP_Seq,Num_Seq,PAM30,BLOSUM,RRM_SN,RRM_Corr,Matching_sseqs
13,SVPHFSDEDKDP,"[0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...","[2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3]",None,None,None,None,None
14,VPHFSDEDKDPE,"[0.0057, 0.0198, 0.0242, 0.0946, 0.0829, 0.126...","[1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3, 5]",None,None,None,None,None
28,SVPHFSEEEKEA,"[0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...","[2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 1]",None,None,None,None,None
29,VPHFSEEEKEAE,"[0.0057, 0.0198, 0.0242, 0.0946, 0.0829, 0.005...","[1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 1, 5]",None,None,None,None,None
43,SVPHFSDEDKDP,"[0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...","[2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3]",None,None,None,None,None
...,...,...,...,...,...,...,...,...
28873,FLRRIRPKLKWD,"[0.0946, 0.0, 0.0959, 0.0959, 0.0, 0.0959, 0.0...","[0, 1, 6, 6, 1, 6, 3, 6, 1, 6, 0, 5]",None,None,None,None,None
28874,LRRIRPKLKWDN,"[0.0, 0.0959, 0.0959, 0.0, 0.0959, 0.0198, 0.0...","[1, 6, 6, 1, 6, 3, 6, 1, 6, 0, 5, 2]",None,None,None,None,None
28875,RRIRPKLKWDNQ,"[0.0959, 0.0959, 0.0, 0.0959, 0.0198, 0.0371, ...","[6, 6, 1, 6, 3, 6, 1, 6, 0, 5, 2, 2]",None,None,None,None,None
28903,YGGFLRRQFKVV,"[0.0516, 0.005, 0.005, 0.0946, 0.0, 0.0959, 0....","[0, 4, 4, 0, 1, 6, 6, 2, 0, 6, 1, 1]",None,None,None,None,None


In [24]:
similarity.get_df_with_binder_subseqs(min_length=3)[SEQS['GRBP5']]

,Sequences,EIIP_Seq,Num_Seq,PAM30,BLOSUM,RRM_SN,RRM_Corr,Matching_sseqs
2589,RGPAESSGESAH,"[0.0959, 0.005, 0.0198, 0.0373, 0.0058, 0.0829...","[6, 4, 3, 1, 5, 2, 2, 4, 5, 2, 1, 6]",None,None,None,None,None
2670,RGPAESSGESAH,"[0.0959, 0.005, 0.0198, 0.0373, 0.0058, 0.0829...","[6, 4, 3, 1, 5, 2, 2, 4, 5, 2, 1, 6]",None,None,None,None,None
2712,RGPAESSGESAL,"[0.0959, 0.005, 0.0198, 0.0373, 0.0058, 0.0829...","[6, 4, 3, 1, 5, 2, 2, 4, 5, 2, 1, 1]",None,None,None,None,None
21485,ASSGESSGMWFG,"[0.0373, 0.0829, 0.0829, 0.005, 0.0058, 0.0829...","[1, 2, 2, 4, 5, 2, 2, 4, 1, 0, 0, 4]",None,None,None,None,None
23968,SGSGESSGMWFG,"[0.0829, 0.005, 0.0829, 0.005, 0.0058, 0.0829,...","[2, 4, 2, 4, 5, 2, 2, 4, 1, 0, 0, 4]",None,None,None,None,None
10340,GFDDPRLFSSSY,"[0.005, 0.0946, 0.1263, 0.1263, 0.0198, 0.0959...","[4, 0, 5, 5, 3, 6, 1, 0, 2, 2, 2, 0]",None,None,None,None,None


In [36]:
def get_PAM30():
    pam30 = similarity.data["PAM30"]
    print(pam30)
    
get_PAM30()

     0    1    2    3    4    5    6    7    8    9   ...   18   19   20   21  \
0   NaN    A    R    N    D    C    Q    E    G    H  ...    W    Y    V    B   
1     A    6   -7   -4   -3   -6   -4   -2   -2   -7  ...  -13   -8   -2   -3   
2     R   -7    8   -6  -10   -8   -2   -9   -9   -2  ...   -2  -10   -8   -7   
3     N   -4   -6    8    2  -11   -3   -2   -3    0  ...   -8   -4   -8    6   
4     D   -3  -10    2    8  -14   -2    2   -3   -4  ...  -15  -11   -8    6   
5     C   -6   -8  -11  -14   10  -14  -14   -9   -7  ...  -15   -4   -6  -12   
6     Q   -4   -2   -3   -2  -14    8    1   -7    1  ...  -13  -12   -7   -3   
7     E   -2   -9   -2    2  -14    1    8   -4   -5  ...  -17   -8   -6    1   
8     G   -2   -9   -3   -3   -9   -7   -4    6   -9  ...  -15  -14   -5   -3   
9     H   -7   -2    0   -4   -7    1   -5   -9    9  ...   -7   -3   -6   -1   
10    I   -5   -5   -5   -7   -6   -8   -5  -11   -9  ...  -14   -6    2   -6   
11    L   -6   -8   -7  -12 

In [51]:
pam30 = pd.read_csv('../src_data/pam30.csv', index_col=0)
p30 = pam30.to_dict()
pam30

,A,R,N,D,C,Q,E,G,H,I,...,W,Y,V,B,J,Z,X,*,-,.
A,6,-7,-4,-3,-6,-4,-2,-2,-7,-5,...,-13,-8,-2,-3,-6,-3,-1,-17,-17,-17
R,-7,8,-6,-10,-8,-2,-9,-9,-2,-5,...,-2,-10,-8,-7,-7,-4,-1,-17,-17,-17
N,-4,-6,8,2,-11,-3,-2,-3,0,-5,...,-8,-4,-8,6,-6,-3,-1,-17,-17,-17
D,-3,-10,2,8,-14,-2,2,-3,-4,-7,...,-15,-11,-8,6,-10,1,-1,-17,-17,-17
C,-6,-8,-11,-14,10,-14,-14,-9,-7,-6,...,-15,-4,-6,-12,-9,-14,-1,-17,-17,-17
Q,-4,-2,-3,-2,-14,8,1,-7,1,-8,...,-13,-12,-7,-3,-5,6,-1,-17,-17,-17
E,-2,-9,-2,2,-14,1,8,-4,-5,-5,...,-17,-8,-6,1,-7,6,-1,-17,-17,-17
G,-2,-9,-3,-3,-9,-7,-4,6,-9,-11,...,-15,-14,-5,-3,-10,-5,-1,-17,-17,-17
H,-7,-2,0,-4,-7,1,-5,-9,9,-9,...,-7,-3,-6,-1,-7,-1,-1,-17,-17,-17
I,-5,-5,-5,-7,-6,-8,-5,-11,-9,8,...,-14,-6,2,-6,5,-6,-1,-17,-17,-17
